In [ ]:
import pandas as pd
import yfinance as yf
from itertools import product

In [ ]:
returns_df = pd.read_excel("/content/Returns.xlsx", sheet_name=0, index_col=0, engine="openpyxl")
marketcap_df = pd.read_excel("/content/Marketcap.xlsx", sheet_name=0, index_col=0, engine="openpyxl")
Beta_df = pd.read_excel("/content/Beta.xlsx", sheet_name=0, index_col=0, engine="openpyxl")
Weigths_df = pd.read_excel("/content/Weigths.xlsx", sheet_name=0, index_col=0, engine="openpyxl")

In [105]:
returns_df.head()

,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-05,-0.010861,0.000915,NaN,NaN,-0.008078,-0.003327,0.006180,0.016446,NaN,0.005402,...,NaN,0.021228,0.060819,NaN,0.003905,NaN,-0.003420,0.031656,-0.001744,NaN
2010-01-06,-0.003555,-0.015805,NaN,NaN,0.005556,-0.004330,0.010631,-0.002122,NaN,-0.002528,...,NaN,-0.011055,-0.013117,NaN,0.008643,NaN,-0.007146,-0.000323,-0.007687,NaN
2010-01-07,-0.001296,-0.001858,NaN,NaN,0.008284,-0.005874,-0.000935,-0.019405,NaN,-0.010456,...,NaN,-0.004469,0.021356,NaN,-0.003142,NaN,-0.000288,0.022941,-0.025000,NaN
2010-01-08,-0.000322,0.006648,NaN,NaN,0.005110,-0.001978,-0.003978,-0.005422,NaN,-0.012488,...,NaN,-0.009209,-0.007165,NaN,-0.004011,NaN,0.000288,-0.021005,-0.003250,NaN
2010-01-11,0.000649,-0.008850,NaN,NaN,0.005087,-0.003100,-0.000940,-0.013083,NaN,0.000649,...,NaN,0.006570,-0.003240,NaN,0.011220,NaN,0.017280,0.022101,0.003261,NaN


In [106]:
returns_df.columns = [col.split(" ")[0] for col in returns_df.columns if col != "Date"]
marketcap_df.columns = [col.split(" ")[0] for col in marketcap_df.columns if col != "Date"]
Beta_df.columns = [col.split(" ")[0] for col in Beta_df.columns if col != "Date"]
Weigths_df.columns = [col.split(" ")[0] for col in Weigths_df.columns if col != "Date"]

In [129]:
tickers = [
    "CAG", "SWKS", "CPB", "DUK", "LOW", "L", "UPS", "PM", "LNT", "GOOG",
    "TER", "MAS", "MRK", "PNW", "KLAC", "WMT", "SO", "CLX", "KEY", "BRK-B",
    "DPZ", "AMD", "MSFT", "AES", "LLY", "META"
]

In [130]:
# Fetch data from 2010 to present
start_date = "2015-01-01"
end_date = "2024-12-31"

In [131]:
# Initialize an empty DataFrame
merged_df = pd.DataFrame()

In [132]:
i = 0
for ticker in tickers:
    temp = yf.Ticker(ticker)
    stock_data = temp.history(start=start_date, end=end_date)[["High", "Low", "Close", "Volume"]]
    stock_data["Ticker"] = ticker
    stock_data.reset_index(inplace=True)  # Ensure Date is included
    stock_data["Date"] = stock_data["Date"].dt.strftime("%Y-%m-%d")  # Format date to only include day, month, and year

    if i == 0:
        merged_df = stock_data.copy()
        i += 1
        print("Step no:", i)
    else:
        merged_df = pd.concat([merged_df, stock_data], ignore_index=True)
        i += 1
        print("Step no:", i)

Step no: 1
Step no: 2
Step no: 3
Step no: 4
Step no: 5
Step no: 6
Step no: 7
Step no: 8
Step no: 9
Step no: 10
Step no: 11
Step no: 12
Step no: 13
Step no: 14
Step no: 15
Step no: 16
Step no: 17
Step no: 18
Step no: 19
Step no: 20
Step no: 21
Step no: 22
Step no: 23
Step no: 24
Step no: 25
Step no: 26


In [133]:
# Remove the index column if it exists
if 'index' in merged_df.columns:
    merged_df.drop(columns=['index'], inplace=True)

In [134]:
# Combine all data into a single DataFrame
df = merged_df

In [135]:
df.dtypes

,0
Date,object
High,float64
Low,float64
Close,float64
Volume,int64
Ticker,object


In [136]:
df.shape

(65390, 6)

In [137]:
df.sample(10)

,Date,High,Low,Close,Volume,Ticker
56818,2020-11-30,207.175477,203.393918,206.509857,33064800,MSFT
33961,2020-01-14,73.151230,72.332611,73.151230,758300,PNW
915,2018-08-21,29.783622,29.136152,29.625702,6310300,CAG
5664,2017-07-11,40.258866,39.740714,40.125404,2251900,CPB
38567,2018-05-08,25.570590,25.293589,25.537827,19654200,WMT
40097,2024-06-06,67.099649,66.433024,66.811111,13256000,WMT
30179,2024-12-30,72.663495,71.468375,72.225281,821800,MAS
62812,2024-10-01,888.265502,875.907278,881.498352,2455200,LLY
54924,2023-05-19,107.290001,104.620003,105.820000,67830600,AMD
38077,2016-05-26,20.316600,20.094592,20.165749,26081700,WMT


In [144]:
row_counts = df.groupby("Ticker").size()
print(row_counts)

Ticker
AES      2515
AMD      2515
BRK-B    2515
CAG      2515
CLX      2515
CPB      2515
DPZ      2515
DUK      2515
GOOG     2515
KEY      2515
KLAC     2515
L        2515
LLY      2515
LNT      2515
LOW      2515
MAS      2515
META     2515
MRK      2515
MSFT     2515
PM       2515
PNW      2515
SO       2515
SWKS     2515
TER      2515
UPS      2515
WMT      2515
dtype: int64


In [143]:
df.isnull().sum()

,0
Date,0
High,0
Low,0
Close,0
Volume,0
Ticker,0


In [142]:
# Save to CSV
df.to_csv("Yfin_stock_data.csv", index=False)